<a href="https://colab.research.google.com/github/Hammad400/Advance_AI_Learning/blob/main/courses/udacity_intro_to_tensorflow_for_deep_learning/l09c04_nlp_embeddings_and_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Authors.

In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Word Embeddings and Sentiment

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l09c04_nlp_embeddings_and_sentiment.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l09c04_nlp_embeddings_and_sentiment.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this colab, you'll work with word embeddings and train a basic neural network to predict text sentiment. At the end, you'll be able to visualize how the network sees the related sentiment of each word in the dataset.

## Import TensorFlow and related functions

In [3]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Get the dataset

We're going to use a dataset containing Amazon and Yelp reviews, with their related sentiment (1 for positive, 0 for negative). This dataset was originally extracted from [here](https://www.kaggle.com/marklvl/sentiment-labelled-sentences-data-set).

In [4]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2024-12-30 22:21:32--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 142.251.2.102, 142.251.2.100, 142.251.2.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P [following]
--2024-12-30 22:21:32--  https://drive.usercontent.google.com/download?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.101.132, 2607:f8b0:4023:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.101.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 127831 (125K) [application/octet-stream]
Saving to: ‘/tmp/sentiment.csv’

/tmp/sentiment.csv  100%[===================>] 124.83K  --.-KB/s    in 0.04s   

2024-12-30 22:21:35 (2.96 MB/s

In [5]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('/tmp/sentiment.csv')

sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [6]:
dataset.shape

(1992, 3)

## Tokenize the dataset

Tokenize the dataset, including padding and OOV

In [7]:
# vocab_size = 100
# vocab_size = 500
vocab_size = 1000
# vocab_size = 3000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type,
                       truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length,
                               padding=padding_type, truncating=trunc_type)

In [8]:
type(word_index)

dict

In [9]:
len(word_index)

2834

In [10]:
word_index

{'<OOV>': 1,
 'the': 2,
 'and': 3,
 'i': 4,
 'it': 5,
 'a': 6,
 'is': 7,
 'to': 8,
 'this': 9,
 'was': 10,
 'of': 11,
 'my': 12,
 'for': 13,
 'not': 14,
 'phone': 15,
 'in': 16,
 'very': 17,
 'with': 18,
 'great': 19,
 'good': 20,
 'on': 21,
 'that': 22,
 'have': 23,
 'you': 24,
 'had': 25,
 'so': 26,
 'but': 27,
 'are': 28,
 'food': 29,
 'service': 30,
 'be': 31,
 'as': 32,
 'place': 33,
 'all': 34,
 'like': 35,
 'quality': 36,
 'time': 37,
 'product': 38,
 'one': 39,
 'at': 40,
 'we': 41,
 'well': 42,
 'they': 43,
 'would': 44,
 'back': 45,
 'if': 46,
 'headset': 47,
 'works': 48,
 'battery': 49,
 'really': 50,
 "don't": 51,
 'sound': 52,
 'your': 53,
 'use': 54,
 'best': 55,
 'up': 56,
 'from': 57,
 'has': 58,
 'just': 59,
 'an': 60,
 "i've": 61,
 'will': 62,
 'no': 63,
 'were': 64,
 'only': 65,
 'also': 66,
 'me': 67,
 'there': 68,
 'ever': 69,
 'than': 70,
 "it's": 71,
 'out': 72,
 'nice': 73,
 'ear': 74,
 'go': 75,
 'excellent': 76,
 'when': 77,
 'first': 78,
 'work': 79,
 'recom

In [11]:
list(word_index.items())[-1]

('philadelphia', 2834)

In [12]:
sequences

[[26,
  68,
  7,
  63,
  173,
  13,
  67,
  8,
  218,
  5,
  16,
  82,
  16,
  2,
  197,
  363,
  4,
  75,
  109,
  6,
  1],
 [20, 90, 76, 364],
 [19, 13, 2, 613],
 [1, 8, 137, 13, 614, 810, 94, 70, 811, 130, 1, 198],
 [2, 505, 7, 19],
 [4, 23, 8, 1, 2, 218, 8, 91, 5, 8, 422, 56, 131, 8, 91, 423, 199],
 [46,
  24,
  23,
  238,
  1,
  85,
  238,
  1,
  615,
  125,
  1,
  2,
  616,
  11,
  506,
  365,
  11,
  116,
  39,
  109,
  39],
 [46, 24, 28, 424, 1, 24, 322, 23, 9],
 [1, 8, 174, 4, 812, 12, 119],
 [95, 6, 141, 11, 119, 3, 37],
 [3, 2, 52, 36, 7, 19],
 [148, 10, 17, 219, 77, 157, 57, 2, 366, 49, 8, 2, 813, 49],
 [46,
  2,
  149,
  64,
  1,
  109,
  6,
  1,
  186,
  1,
  4,
  425,
  8,
  814,
  1,
  617,
  3,
  1,
  52,
  57,
  2,
  47],
 [17, 20, 36, 367],
 [2, 220, 7, 17, 1, 32, 2, 74, 507, 7, 14, 17, 142, 40, 34],
 [221, 80, 13, 96, 39, 289, 58, 6, 508, 815, 15],
 [4, 816, 323, 105, 14, 31, 1],
 [26, 143, 26, 20],
 [48, 19],
 [5, 1, 200, 33, 16, 6, 173, 22, 222, 24, 1, 158, 175, 2

In [13]:
training_sentences[0]
# len(training_sentences[0])
# len(training_sentences[0].split())

'So there is no way for me to plug it in here in the US unless I go by a converter.'

In [14]:
sequences[0]

[26,
 68,
 7,
 63,
 173,
 13,
 67,
 8,
 218,
 5,
 16,
 82,
 16,
 2,
 197,
 363,
 4,
 75,
 109,
 6,
 1]

In [15]:
len(sequences[0])

21

In [16]:
padded[0]

array([ 26,  68,   7,  63, 173,  13,  67,   8, 218,   5,  16,  82,  16,
         2, 197, 363,   4,  75, 109,   6,   1,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [17]:
padded[0].ndim

1

In [47]:
len(padded[0])

100

In [24]:
padded.shape

(1593, 100)

## Review a Sequence

Let's quickly take a look at one of the padded sequences to ensure everything above worked appropriately.

In [60]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

good case excellent <OOV> ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Good case Excellent value.


## Train a Basic Sentiment Model with Embeddings

In [ ]:
# Build a basic sentiment network
# Note the embedding layer is first,
# and the output is only 1 node as it is either 0 or 1 (negative or positive)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

## Get files for visualizing the network

The code below will download two files for visualizing how your network "sees" the sentiment related to each word. Head to http://projector.tensorflow.org/ and load these files, then click the "Sphereize" checkbox.

In [ ]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

In [ ]:
import io

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
# Download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

## Predicting Sentiment in New Reviews

Now that you've trained and visualized your network, take a look below at how we can predict sentiment in new reviews the network has never seen before.

In [ ]:
# Use the model to predict a review
fake_reviews = ['I love this phone', 'I hate spaghetti',
                'Everything was cold',
                'Everything was hot exactly as I wanted',
                'Everything was green',
                'the host seated us immediately',
                'they gave us free chocolate cake',
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes',
                'does not work when I stand on my head']

print(fake_reviews)

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

# Try adding reviews of your own
# Add some negative words (such as "not") to the good reviews and see what happens
# For example:
# they gave us free chocolate cake and did not charge us